Задание 2 Замена сверточных слоёв на слои с максимальным пулингом и средним пулингом

1) Подключение библиотек

In [28]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset, DataLoader

2) Настройка вычислений

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

torch.manual_seed(42)

device: cuda


3) Загрузка подготовленного датасета

In [30]:
DATA_PATH = r"..\data\prepared\notmnist_28.npz"
assert os.path.exists(DATA_PATH), f"File not found: {DATA_PATH}"

d = np.load(DATA_PATH)

X_train = torch.tensor(d["X_train"], dtype=torch.float32)
y_train = torch.tensor(d["y_train"], dtype=torch.long)

X_val = torch.tensor(d["X_val"], dtype=torch.float32)
y_val = torch.tensor(d["y_val"], dtype=torch.long)

X_test = torch.tensor(d["X_test"], dtype=torch.float32)
y_test = torch.tensor(d["y_test"], dtype=torch.long)

print("train:", X_train.shape, y_train.shape)
print("val:  ", X_val.shape, y_val.shape)
print("test: ", X_test.shape, y_test.shape)

train: torch.Size([415752, 1, 28, 28]) torch.Size([415752])
val:   torch.Size([46194, 1, 28, 28]) torch.Size([46194])
test:  torch.Size([13649, 1, 28, 28]) torch.Size([13649])


4) Фомирование DataLoader

In [31]:
BATCH_SIZE = 128

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader   = DataLoader(TensorDataset(X_val, y_val),     batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_loader  = DataLoader(TensorDataset(X_test, y_test),   batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

5) Определение архитектуры нейронной сети (2 максимальных пула + 2 средних пула, активация кусочко-линейная)

In [32]:
class SimplePoolNet(nn.Module):
    
    def __init__(self, pool_type="max"):
        super().__init__()

        if pool_type == "max":
            self.layer1 = nn.MaxPool2d(2, 2)
            self.layer2 = nn.MaxPool2d(2, 2)
        elif pool_type == "avg":
            self.layer1 = nn.AvgPool2d(2, 2)
            self.layer2 = nn.AvgPool2d(2, 2)
        else:
            raise ValueError("pool_type must be 'max' or 'avg'")

        self.act = nn.ReLU()
        self.fc = nn.Linear(1 * 7 * 7, 10)

    def forward(self, x):
        x = self.act(self.layer1(x))
        x = self.act(self.layer2(x))
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

6) Проверка корректности размерностей

In [33]:
model = SimplePoolNet(pool_type="max").to(device)

xb, yb = next(iter(train_loader))
xb = xb.to(device)

out = model(xb)
print("out shape:", out.shape)

out shape: torch.Size([128, 10])


7) Оптимизатор и гиперпараметры обучения

In [34]:
LR = 1e-3
EPOCHS = 30

optimizer = torch.optim.Adam(model.parameters(), lr=LR)

8) Функции обучения и оценки качества

In [35]:
def accuracy(logits, y):
    preds = logits.argmax(dim=1)
    return (preds == y).float().mean().item()


@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    total_loss, total_acc, n = 0.0, 0.0, 0

    for X, y in loader:
        X, y = X.to(device), y.to(device)
        logits = model(X)
        loss = F.cross_entropy(logits, y)

        bs = X.size(0)
        total_loss += loss.item() * bs
        total_acc += accuracy(logits, y) * bs
        n += bs

    return total_loss / n, total_acc / n


def train_one_epoch(model, loader, optimizer):
    model.train()
    total_loss, total_acc, n = 0.0, 0.0, 0

    for X, y in loader:
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        logits = model(X)
        loss = F.cross_entropy(logits, y)
        loss.backward()
        optimizer.step()

        bs = X.size(0)
        total_loss += loss.item() * bs
        total_acc += accuracy(logits, y) * bs
        n += bs

    return total_loss / n, total_acc / n

9) Обучение модели

In [36]:
for epoch in range(1, EPOCHS + 1):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer)
    val_loss, val_acc = evaluate(model, val_loader)

    print(f"Epoch {epoch:02d}: "
          f"train loss={train_loss:.4f}, acc={train_acc:.4f} | "
          f"val loss={val_loss:.4f}, acc={val_acc:.4f}")

Epoch 01: train loss=1.0306, acc=0.7344 | val loss=0.8518, acc=0.7724
Epoch 02: train loss=0.8373, acc=0.7721 | val loss=0.8300, acc=0.7756
Epoch 03: train loss=0.8250, acc=0.7741 | val loss=0.8238, acc=0.7745
Epoch 04: train loss=0.8210, acc=0.7751 | val loss=0.8218, acc=0.7767
Epoch 05: train loss=0.8193, acc=0.7754 | val loss=0.8198, acc=0.7778
Epoch 06: train loss=0.8182, acc=0.7755 | val loss=0.8211, acc=0.7820
Epoch 07: train loss=0.8177, acc=0.7754 | val loss=0.8189, acc=0.7784
Epoch 08: train loss=0.8174, acc=0.7756 | val loss=0.8192, acc=0.7741
Epoch 09: train loss=0.8172, acc=0.7755 | val loss=0.8196, acc=0.7820
Epoch 10: train loss=0.8171, acc=0.7756 | val loss=0.8185, acc=0.7771
Epoch 11: train loss=0.8170, acc=0.7754 | val loss=0.8178, acc=0.7786
Epoch 12: train loss=0.8170, acc=0.7755 | val loss=0.8184, acc=0.7757
Epoch 13: train loss=0.8169, acc=0.7758 | val loss=0.8195, acc=0.7778
Epoch 14: train loss=0.8168, acc=0.7753 | val loss=0.8190, acc=0.7777
Epoch 15: train loss

10) Оценка качества на тестовой выборке

In [37]:
test_loss, test_acc = evaluate(model, test_loader)
print(f"TEST: loss={test_loss:.4f}, acc={test_acc:.4f}")

TEST: loss=0.6167, acc=0.8289


В модели второго задания сверточные слои заменены на пулинг-операции. По результатам обучения видно, что точность на обучающей выборке достигает значения около 0.77 уже на 2–3 эпохе и далее практически не изменяется. Аналогично, точность на валидационной выборке колеблется в диапазоне 0.77–0.78. Это свидетельствует о достижении плато и невозможности существенного улучшения качества при дальнейшем обучении. Причина заключается в том, что пулинг-слои не содержат обучаемых параметров и выполняют только операцию уменьшения размерности, не извлекая информативные признаки изображения. В результате модель обладает недостаточной выразительной способностью и демонстрирует более низкую точность на тестовой выборке 0.83.